In [8]:
from urllib.parse import quote_plus
import sqlalchemy

from pathlib import Path
cwd = Path.cwd()
data_path = cwd / 'data'
import pandas as pd

params = quote_plus(
    """DRIVER={ODBC Driver 13 for SQL Server};SERVER=server-dw01.database.windows.net;DATABASE=DataWarehouseStaging;UID=dw_userlogin;PWD=kljDueK!7"""
)

ENGINE = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)


def dwhQuery(sqlQuery):
    """
    Wrapper for turning SQL query to DWH
    into a pandas dataframe
    :param sqlQuery: SQL query to be processed
    :return: SQL query as a pandas DataFrame
    """
    return pd.read_sql(sql=sqlQuery, con=ENGINE)

In [9]:
SIC_df = dwhQuery('''SELECT * FROM pam.sic_codes''')

In [10]:
df = pd.DataFrame()
for i, code in enumerate(SIC_df['SIC_CODE'].unique()):
    code_df = SIC_df[SIC_df['SIC_CODE'] == code]
    desc_list = []
    for i, row in code_df.iterrows():
        desc_list.append(row['Description'].split())
    desc_list = list(set([item.lower() for sublist in desc_list for item in sublist]))
    desc = ' '.join(desc_list)
    r = pd.DataFrame(data = {'SIC_CODE':code, 'desc':desc}, index = [i])
    df = pd.concat([df, r], sort = True)

In [13]:
from modules.fastSQL import to_sql_fast
to_sql_fast(df, name = 'pam.sic_simple', engine = ENGINE, if_exists = 'replace')

In [17]:
mcc_sic_df = dwhQuery('''SELECT DISTINCT
	t.MCC_Desc
	,s.[desc] AS [SIC_DESC]

FROM ch.companies AS c

LEFT JOIN Accounts AS a
	ON a.[Registered No] = c.[original_CRN]

LEFT JOIN Opportunities AS o
	ON o.[Account ID] = a.[Account ID]
		AND o.[Opportunity Record Type] NOT LIKE '%renewal%'

LEFT JOIN marketing.WP_Attri_Mailers_bkp AS t
	ON t.[Opportunity ID] = o.[Opportunity ID]

LEFT JOIN pam.sic_simple AS s
	ON s.SIC_CODE = [sic_codes__0]

WHERE t.MCC_Desc IS NOT NULL
	AND s.[desc] IS NOT NULL''')

In [18]:
mcc_sic_df

,MCC_Desc,SIC_DESC
0,ADVERTISING SERVICES,imprinting and public searching real-time repo...
1,ADVERTISING SERVICES,on-line newspaper advertising publishing
2,ADVERTISING SERVICES,peripheral and installation activities compute...
3,ADVERTISING SERVICES,sites structures and marketing showroom of con...
4,AIRPORTS FLYING FIELDS TERMNLS,regular taxi transport and pleasure for servic...
...,...,...
1592,WOMENS READY TO WEAR STORES,substitutes wallets and leather saddlery made ...
1593,WOMENS READY TO WEAR STORES,tapestry piece and thread table-cloths fabric ...
1594,WRECKING & SALVAGE YARDS,"clay and of pallets, prepared load glass in ti..."
1595,WRECKING & SALVAGE YARDS,clay exporter and yard mastics props metal lin...


In [19]:
dict_df = pd.DataFrame()
for i, mcc in enumerate(mcc_sic_df['MCC_Desc'].unique()):
    mcc_df = mcc_sic_df[mcc_sic_df['MCC_Desc'] == mcc]
    sic_desc_list = []
    for i, row in mcc_df.iterrows():
        sic_desc_list.append(row['SIC_DESC'].split())
    sic_desc_list = list(set([item.lower() for sublist in sic_desc_list for item in sublist]))
    sic_desc = ' '.join(sic_desc_list)
    r = pd.DataFrame(data = {'MCC_DESC':mcc, 'SIC_DESC':sic_desc}, index = [i])
    dict_df = pd.concat([dict_df, r], sort = True)

In [26]:
dict_df['desc'] = dict_df['MCC_DESC'] + ' ' + dict_df['SIC_DESC']

In [27]:
to_sql_fast(dict_df, name = 'pam.mcc_sic_dict', engine = ENGINE, if_exists = 'replace')